# Physics-Informed Decline Curve Analysis

This notebook demonstrates the physics-informed capabilities of the `decline_curve` library.

## Learning Objectives

By the end of this notebook, you will be able to:
- Calculate PVT properties (formation volume factors, solution GOR, viscosities)
- Model Inflow Performance Relationships (IPR)
- Use material balance for decline forecasting
- Apply physics-informed constraints to forecasts

## Table of Contents
1. [Setup](#setup)
2. [PVT Correlations](#pvt)
3. [Inflow Performance Relationships (IPR)](#ipr)
4. [Material Balance Forecasting](#material-balance)
5. [Physics-Informed Constraints](#constraints)

## 1. Setup {#setup}

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from decline_curve import dca
from decline_curve.pvt import calculate_pvt_properties, standing_rs, standing_bo, gas_z_factor, gas_fvf
from decline_curve.ipr import linear_ipr, vogel_ipr, fetkovich_ipr
from decline_curve.physics_informed import MaterialBalanceParams, material_balance_forecast, apply_physics_constraints
from decline_curve.plot import minimal_style

minimal_style()
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ All imports successful!")

## 2. PVT Correlations {#pvt}

PVT (Pressure-Volume-Temperature) properties are essential for accurate reservoir engineering calculations.

In [ ]:
# Define reservoir conditions
pressure = np.linspace(1000, 5000, 50)
temperature = 200.0
api_gravity = 35.0
gas_gravity = 0.65

# Calculate PVT properties
rs_values = [standing_rs(p, temperature, api_gravity, gas_gravity) for p in pressure]
bo_values = [standing_bo(rs_values[i], temperature, api_gravity, gas_gravity) for i, p in enumerate(pressure)]
z_values = [gas_z_factor(p, temperature, gas_gravity, method='standing') for p in pressure]
bg_values = [gas_fvf(p, temperature, z_values[i]) for i, p in enumerate(pressure)]

# Comprehensive PVT calculation
pvt_props = calculate_pvt_properties(pressure[25], temperature, api_gravity, gas_gravity, method='standing')

print("=" * 70)
print("PVT PROPERTIES")
print("=" * 70)
print(f"Solution GOR (Rs): {pvt_props.Rs:.2f} SCF/STB")
print(f"Oil FVF (Bo): {pvt_props.Bo:.4f} RB/STB")
print(f"Gas Z-Factor: {pvt_props.Z:.4f}")
print(f"Gas FVF (Bg): {pvt_props.Bg:.6f} RB/SCF")
print("=" * 70)

## 3. Inflow Performance Relationships (IPR) {#ipr}

In [ ]:
# Define well parameters
reservoir_pressure = 4000.0
wellbore_pressure = np.linspace(500, 4000, 50)
productivity_index = 2.0
bubble_point = 2000.0

# Calculate IPR curves
linear_rates = [linear_ipr(reservoir_pressure, pwf, productivity_index) for pwf in wellbore_pressure]
vogel_rates = [vogel_ipr(reservoir_pressure, pwf, max_rate=productivity_index * reservoir_pressure / 1.8, productivity_index=productivity_index, bubble_point_pressure=bubble_point) for pwf in wellbore_pressure]
fetkovich_rates = [fetkovich_ipr(reservoir_pressure, pwf, q_max=productivity_index * reservoir_pressure, n=0.8) for pwf in wellbore_pressure]

print("=" * 70)
print("IPR MODEL COMPARISON")
print("=" * 70)
print(f"Maximum Rates:")
print(f"  Linear IPR: {max(linear_rates):.1f} bbl/day")
print(f"  Vogel IPR: {max(vogel_rates):.1f} bbl/day")
print(f"  Fetkovich IPR: {max(fetkovich_rates):.1f} bbl/day")
print("=" * 70)

## 4. Material Balance Forecasting {#material-balance}

In [ ]:
# Generate sample production data
dates = pd.date_range('2020-01-01', periods=36, freq='MS')
production = pd.Series(1000 * np.exp(-0.01 * np.arange(36)), index=dates, name='oil')

# Material balance forecast
mb_params = MaterialBalanceParams(N=1e6, Boi=1.2, pi=5000.0, drive_mechanism='solution_gas')
forecast_mb = material_balance_forecast(production, material_balance_params=mb_params, horizon=24)

# Compare with Arps
forecast_arps = dca.forecast(production, model='arps', kind='hyperbolic', horizon=24)

print("=" * 70)
print("MATERIAL BALANCE FORECASTING")
print("=" * 70)
print(f"Material Balance first forecast: {forecast_mb.iloc[0]:.1f} bbl/month")
print(f"Arps first forecast: {forecast_arps.iloc[len(production)]:.1f} bbl/month")
print("=" * 70)

## 5. Physics-Informed Constraints {#constraints}

In [ ]:
# Apply physics constraints
unconstrained = np.array([100, 120, 110, 90, 80, 70, -5, 60])
historical = np.array([100, 95, 90])

constrained = apply_physics_constraints(
    unconstrained,
    historical=historical,
    min_rate=0.0,
    max_increase=0.1,
    enforce_decline=False
)

print("=" * 70)
print("PHYSICS-INFORMED CONSTRAINTS")
print("=" * 70)
print(f"Unconstrained: {unconstrained}")
print(f"Constrained: {constrained}")
print(f"✓ Non-negative: {all(constrained >= 0)}")
print("=" * 70)